In [233]:
import requests
import json
import pandas as pd 
import numpy as np
from collections import defaultdict
import random

In [3]:
x_auth_token = "f5a4370d154f58a4b668798ebf98da56"
url = "https://huqeyhi95c.execute-api.ap-northeast-2.amazonaws.com/prod"

In [4]:
headers = {
    "X-Auth-Token" : x_auth_token,
    "Content-Type" : "application/json"
}

In [12]:
def startProb(url, prob, headers = {"X-Auth-Token" : x_auth_token,"Content-Type" : "application/json"}):
    start_url = url + '/start'
    params = {"problem" : prob}
    res = requests.post(start_url, headers = headers, params = params)
    return res.json()['auth_key']

In [119]:
def getWaitLine(url, auth_key):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    res = requests.get(url + '/waiting_line', headers = headers)
    return res.json()['waiting_line']

In [102]:
def getGameResult(url, auth_key):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    res = requests.get(url + '/game_result', headers = headers)
    return res.json()['game_result']
getGameResult(url, auth_key)

[{'taken': 23, 'win': 4, 'lose': 26}]

In [68]:
def getUserInfo(url, auth_key):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    res = requests.get(url + '/user_info', headers = headers)
    return res.json()['user_info']

In [44]:
def putMatch(url, auth_key, pairs):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    params = {
        "pairs" : pairs
    }
    res = requests.put(url + '/match', headers = headers, data = json.dumps(params))
    return res.json()

putMatch(url, auth_key, [])

{'status': 'ready', 'time': 4}

In [38]:
def putChangeGrade(url, auth_key, commands):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    params = {
        "commands" : commands
    }
    res = requests.put(url + '/change_grade', headers = headers, data = json.dumps(params))

    return res.json()
putChangeGrade(url, auth_key, [{"id" : 1, "grade" : 1900},])

{'status': 'ready'}

In [39]:
def getScore(url, auth_key):
    headers = {'Authorization' : auth_key,
              'Content-Type' : 'application/json'}
    res = requests.get(url + '/score', headers = headers)
    return res.json()
getScore(url, auth_key)

{'message': '종료된 이후 score 조회가 가능합니다.'}

In [468]:
def findMatch(wait_line, userGrade, game_log, turn):
    pairs = []
    wait_users = [line['id']  for line in wait_line]
    
    users_from = {line['id']:line['from']  for line in wait_line}
    while wait_users : 
        diff = 99999999
        user1 = wait_users[0]
        if len(wait_users) == 1 :
            break
        fight = 0
        grade1 = userGrade[user1-1]['grade']
        
        for user2 in wait_users :
            if user2 == user1 : continue
#             if (user2 in game_log[user1]['win']) and (turn - users_from[user2] < 1): continue
            grade2 = userGrade[user2-1]['grade']
            if abs(grade1-grade2) < diff :
                diff = abs(grade1-grade2)
                fight = user2
        pairs.append([user1, fight])
        wait_users.remove(user1)
        if fight == 0 :
            pairs.remove([user1, fight])
        else :
            wait_users.remove(fight)
            
    return pairs

In [469]:
def changeUserGrade(exist, userInfo, game_log, num):
    points = []
    for i in range(1, num+1):
        lose_point = 0
        for lose_log in game_log[i]['lose']:
            lose_point += (num - userInfo[lose_log-1]['grade'])
        win_point = 0
        for win_log in game_log[i]['win']:
            win_point += (num - userInfo[win_log-1]['grade'])
        i_point = win_point - lose_point
        points.append([i, i_point])
    
    points = sorted(points, key = lambda x : x[1])
    
    new_commands = [{'id' : i , 'grade' : grade} for grade,(i,point) in enumerate(points, 1)]
    
    return new_commands

In [470]:
# Set All Users Grade 
num = 30
auth_key = startProb(url, 1)
random_init = random.choices([i for i in range(1,num+1)], k = num)
commands = [{"id" : i, "grade" : grade} for i, grade in enumerate(random_init, 1)]
putChangeGrade(url, auth_key, commands)
putMatch(url, auth_key, [])

# Random Match
userInfo = getUserInfo(url, auth_key)
game_log = {i : {'win': [] ,'lose': [], 'taken' : [] } for i in range(1, num+1)}

putChangeGrade(url, auth_key, commands)
putMatch(url, auth_key, [])
wait_line = getWaitLine(url, auth_key)
pairs = findMatch(wait_line, userInfo, game_log, 0)

In [471]:
while True:
    turn = putMatch(url, auth_key, pairs = pairs)
    time = turn['time']
    print(time)
    userInfo = getUserInfo(url, auth_key)
    if time == 595 : 
        commands = changeUserGrade(commands, userInfo, game_log, num)
        putChangeGrade(url, auth_key, commands)
        putMatch(url, auth_key, [])
        break
    game_result = getGameResult(url, auth_key)
    if game_result : 
        for gs in game_result :
            game_log[gs['win']]['win'].append(gs['lose'])
            game_log[gs['lose']]['lose'].append(gs['win'])
            game_log[gs['win']]['taken'].append(gs['taken'])
            game_log[gs['lose']]['taken'].append(gs['taken'])
        commands = changeUserGrade(commands, userInfo, game_log, num)
    putChangeGrade(url, auth_key, commands)
    
    userInfo = getUserInfo(url, auth_key)
    wait_line = getWaitLine(url, auth_key)
    if wait_line :
        pairs = findMatch(wait_line, userInfo, game_log, time)
    else :
        pairs = []
    

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [467]:
userInfo

[{'id': 1, 'grade': 25},
 {'id': 2, 'grade': 16},
 {'id': 3, 'grade': 19},
 {'id': 4, 'grade': 26},
 {'id': 5, 'grade': 7},
 {'id': 6, 'grade': 5},
 {'id': 7, 'grade': 21},
 {'id': 8, 'grade': 14},
 {'id': 9, 'grade': 23},
 {'id': 10, 'grade': 13},
 {'id': 11, 'grade': 22},
 {'id': 12, 'grade': 20},
 {'id': 13, 'grade': 15},
 {'id': 14, 'grade': 3},
 {'id': 15, 'grade': 12},
 {'id': 16, 'grade': 28},
 {'id': 17, 'grade': 24},
 {'id': 18, 'grade': 17},
 {'id': 19, 'grade': 9},
 {'id': 20, 'grade': 1},
 {'id': 21, 'grade': 29},
 {'id': 22, 'grade': 30},
 {'id': 23, 'grade': 27},
 {'id': 24, 'grade': 2},
 {'id': 25, 'grade': 6},
 {'id': 26, 'grade': 4},
 {'id': 27, 'grade': 8},
 {'id': 28, 'grade': 18},
 {'id': 29, 'grade': 11},
 {'id': 30, 'grade': 10}]

In [472]:
getScore(url, auth_key)

{'status': 'finished',
 'efficiency_score': '99.1763',
 'accuracy_score1': '46.0',
 'accuracy_score2': '54.6904',
 'score': '200.1695'}